# AIM : Download VoxCeleb1, extract audio samples and evaluate their quality to build the metadata csv

## Libraries

In [ ]:
!pip install pesq
!pip install pystoi
!pip install librosa
!pip install torchaudio
!pip install pandas
!pip install tqdm
!pip install datasets soundfile


  Using cached pesq-0.0.4.tar.gz (38 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pesq: filename=pesq-0.0.4-cp310-cp310-win_amd64.whl size=116658 sha256=35b22b4d8f465358ca592f2e16aef026605bb5c9ecd12fd481eaf3f0690aa48c
  Stored in directory: c:\users\leami\appdata\local\pip\cache\wheels\c5\4e\2c\251524370c0fdd659e99639a0fbd0ca5a782c3aafcd456b28d
Successfully built pesq



[notice] A new release of pip is available: 25.0 -> 25.3
[notice] To update, run: C:\Users\leami\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


     ---------------------------------------- 12.9/12.9 MB 7.5 MB/s eta 0:00:00
     ---------------------------------------- 41.3/41.3 MB 7.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 260.7/260.7 kB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 71.3/71.3 kB 3.8 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 8.8 MB/s eta 0:00:00
     -------------------------------------- 173.6/173.6 kB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 8.9/8.9 MB 8.1 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 9.3 MB/s eta 0:00:00
     -------------------------------------- 308.4/308.4 kB 9.6 MB/s eta 0:00:00
  Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
     ---------------------------------------- 44.6/44.6 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 66.5/66.5 kB ? eta 0:00:00
     ---------------------------------------- 38.1/38.1 MB 8.5 MB/s eta 0:00:00
     ---------------------------------------- 64.7/64.7 kB ? eta 0:00:00
     ------------------------------------- 182.8/182.8 kB 10.8 MB/s eta 0:00:00
     ---


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 663.9/663.9 kB 2.8 MB/s eta 0:00:00
     -------------------------------------- 109.3/109.3 MB 6.9 MB/s eta 0:00:00
     ------------------------------------- 201.0/201.0 kB 11.9 MB/s eta 0:00:00
     ---------------------------------------- 134.9/134.9 kB ? eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 8.4 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 7.6 MB/s eta 0:00:00
     ------------------------------------- 536.2/536.2 kB 11.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import torch  
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from pesq import pesq
from pystoi import stoi
import librosa.core
import soundfile as sf
import torchaudio
import random
import shutil

# 1. load dataset

We are going to use a subset of LibriSpeech dataset available via torchaudio library.

In [ ]:
# Use dev-clean (smallest subset) and handle the loading more carefully
DATA_DIR = "./data/librispeech_subset"
RAW_DIR = os.path.join(DATA_DIR, "raw")
os.makedirs(RAW_DIR, exist_ok=True)

try:
    # Download dev-clean (smallest subset, ~337MB)
    dataset = torchaudio.datasets.LIBRISPEECH(
        root="./data",
        url="dev-clean",  # smallest subset
        download=True
    )
    
    # Take a small random sample
    N_SAMPLES = 50  # reduced from 100 to 50 for testing
    total_files = len(dataset)
    sampled_indices = random.sample(range(total_files), min(N_SAMPLES, total_files))
    
    print(f"Saving {len(sampled_indices)} samples to {RAW_DIR}")
    for idx in tqdm(sampled_indices):
        try:
            # Load audio
            waveform, sample_rate, _, _, _, _ = dataset[idx]
            
            # Save as WAV
            filename = f"librispeech_sample_{idx:04d}.wav"
            filepath = os.path.join(RAW_DIR, filename)
            torchaudio.save(filepath, waveform, sample_rate)
            
        except Exception as e:
            print(f"Error processing file {idx}: {e}")
            continue
            
    print(f"Dataset prepared in {RAW_DIR}")
    
except Exception as e:
    print(f"Failed to load dataset: {e}")
    print("Please check your internet connection and try again.")

100%|██████████| 322M/322M [00:33<00:00, 10.0MB/s] 


Saving 50 samples to ./data/librispeech_subset\raw


  4%|▍         | 2/50 [00:00<00:02, 17.38it/s]

Error processing file 1404: Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6, and 7 on all platforms, and 8 on Mac and Linux.
          2. The PyTorch version (2.9.0+cpu) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were raised as we tried to load libtorchcodec:
        
[start of libtorchcodec loading traceback]
FFmpeg version 8: Could not load this library: C:\Users\leami\Documents\projet_git\GaussBuster\.venv\Lib\site-packages\torchcodec\libtorchcodec_core8.dll
FFmpeg version 7: Could not load this library: C:\Users\leami\Documents\projet_git\GaussBuster\.venv\Lib\site-packages\torchcodec\libtorchcodec_core7.dll
FFmpeg ver

 10%|█         | 5/50 [00:00<00:02, 22.17it/s]

...

Error processing file 1809: Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6, and 7 on all platforms, and 8 on Mac and Linux.
          2. The PyTorch version (2.9.0+cpu) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were raised as we tried to load libtorchcodec:
        
[start of libtorchcodec loading traceback]
FFmpeg version 8: Could not load this library: C:\Users\leami\Documents\projet_git\GaussBuster\.venv\Lib\site-packages\torchcodec\libtorchcodec_core8.dll
FFmpeg version 7: Could not load this library: C:\Users\leami\Documents\projet_git\GaussBuster\.venv\Lib\site-packages\torchcodec\libtorchcodec_core7.dll
FFmpeg ver

 92%|█████████▏| 46/50 [00:02<00:00, 22.53it/s]

Error processing file 218: Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6, and 7 on all platforms, and 8 on Mac and Linux.
          2. The PyTorch version (2.9.0+cpu) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were raised as we tried to load libtorchcodec:
        
[start of libtorchcodec loading traceback]
FFmpeg version 8: Could not load this library: C:\Users\leami\Documents\projet_git\GaussBuster\.venv\Lib\site-packages\torchcodec\libtorchcodec_core8.dll
FFmpeg version 7: Could not load this library: C:\Users\leami\Documents\projet_git\GaussBuster\.venv\Lib\site-packages\torchcodec\libtorchcodec_core7.dll
FFmpeg vers

100%|██████████| 50/50 [00:02<00:00, 21.52it/s]

Error processing file 1655: Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6, and 7 on all platforms, and 8 on Mac and Linux.
          2. The PyTorch version (2.9.0+cpu) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were raised as we tried to load libtorchcodec:
        
[start of libtorchcodec loading traceback]
FFmpeg version 8: Could not load this library: C:\Users\leami\Documents\projet_git\GaussBuster\.venv\Lib\site-packages\torchcodec\libtorchcodec_core8.dll
FFmpeg version 7: Could not load this library: C:\Users\leami\Documents\projet_git\GaussBuster\.venv\Lib\site-packages\torchcodec\libtorchcodec_core7.dll
FFmpeg ver

# 2. Prepare audiofiles

In [ ]:
AUDIO_DIR = RAW_DIR
files = librosa.util.find_files(AUDIO_DIR, ext=["wav"])

print(f"{len(files)} files found.")

2000 files found.


# 3. Calculate audio quality metrics

In [ ]:
def signal_to_noise_ratio(clean, noisy):
    """Calculate Signal-to-Noise Ratio in dB."""
    noise = noisy - clean
    signal_power = torch.mean(clean ** 2)
    noise_power = torch.mean(noise ** 2)
    snr = 10 * torch.log10(signal_power / (noise_power + 1e-8))
    return snr.item()

In [ ]:
# thresholds
MIN_DURATION = 2.0    # seconds
PESQ_MIN = 3.0
STOI_MIN = 0.9
SNR_MIN = 20.0        # dB

results = []
TARGET_SR = 16000  # PESQ requires 8000 or 16000 Hz

for f in tqdm(files[:100]):  # process only 100 files for testing
    try:
        # Load and resample to required rate for PESQ
        y, sr = librosa.load(f, sr=TARGET_SR, mono=True)
        if len(y) < sr * MIN_DURATION:
            continue  # ignore too short

        # Convert to torch tensor
        y = torch.from_numpy(y).float()
        
        # RMS normalization
        y = y / (torch.abs(y).max() + 1e-8)

        # Create degraded version with noise
        noise = torch.randn_like(y) * 0.03
        degraded = y + noise

        # Calculate metrics (using resampled audio)
        clean_np = y.numpy()
        degraded_np = degraded.numpy()
        pesq_score = pesq(TARGET_SR, clean_np, degraded_np, "wb")  # use wb mode for 16kHz
        stoi_score = stoi(clean_np, degraded_np, TARGET_SR, extended=False)
        snr_score = signal_to_noise_ratio(y, degraded)

        results.append({
            "file": f,
            "duration_s": len(y) / TARGET_SR,
            "pesq": pesq_score,
            "stoi": stoi_score, 
            "snr": snr_score
        })
    except Exception as e:
        print(f"Error processing {f}: {e}")

df = pd.DataFrame(results)


 16%|█▌        | 16/100 [00:03<00:17,  4.87it/s]

Error processing c:\Users\leami\Documents\projet_git\GaussBuster\data\ESC-50-master\audio\1-110537-A-22.wav: b'No utterances detected'


 51%|█████     | 51/100 [00:11<00:08,  5.56it/s]

Error processing c:\Users\leami\Documents\projet_git\GaussBuster\data\ESC-50-master\audio\1-16746-A-15.wav: b'No utterances detected'


 56%|█████▌    | 56/100 [00:12<00:08,  4.97it/s]

...

 76%|███████▌  | 76/100 [00:16<00:03,  6.10it/s]

Error processing c:\Users\leami\Documents\projet_git\GaussBuster\data\ESC-50-master\audio\1-18527-A-44.wav: b'No utterances detected'
Error processing c:\Users\leami\Documents\projet_git\GaussBuster\data\ESC-50-master\audio\1-18527-B-44.wav: b'No utterances detected'


 92%|█████████▏| 92/100 [00:20<00:01,  4.61it/s]

Error processing c:\Users\leami\Documents\projet_git\GaussBuster\data\ESC-50-master\audio\1-19872-A-36.wav: b'No utterances detected'
Error processing c:\Users\leami\Documents\projet_git\GaussBuster\data\ESC-50-master\audio\1-19872-B-36.wav: b'No utterances detected'


100%|██████████| 100/100 [00:22<00:00,  4.42it/s]


# 4. Filter and save metadata CSV

In [ ]:
df_filtered = df[
    (df["pesq"] < PESQ_MIN) |
    (df["stoi"] < STOI_MIN) |
    (df["snr"] < SNR_MIN)
].reset_index(drop=True)

print(f"Low quality samples: {len(df_filtered)} samples kept.")

os.makedirs(DATA_DIR, exist_ok=True)
metadata_path = os.path.join(DATA_DIR, "metadata.csv")
df_filtered.to_csv(metadata_path, index=False)
print(f"Metadata saved: {metadata_path}")

# Display first few entries
print(df_filtered.head())

Low quality samples: 90 samples kept.
Metadata saved: ./data/ESC-50-processed\metadata.csv
                                                file  duration_s      pesq  \
0  c:\Users\leami\Documents\projet_git\GaussBuste...         5.0  1.328120   
1  c:\Users\leami\Documents\projet_git\GaussBuste...         5.0  2.290073   
2  c:\Users\leami\Documents\projet_git\GaussBuste...         5.0  4.407367   
3  c:\Users\leami\Documents\projet_git\GaussBuste...         5.0  1.082095   
4  c:\Users\leami\Documents\projet_git\GaussBuste...         5.0  3.150719   

       stoi        snr  
0  0.000010   2.922618  
1  0.726029  11.960645  
2  0.965592  18.561106  
3  0.967487  19.051893  
4  0.581275  10.958883  
